In [30]:
import panel as pn
pn.extension('tabulator')
pn.extension('plotly')

import base64
import datetime
import io

import plotly.graph_objects as go
import pgeocode
import geopandas as gpd
from geopandas import GeoDataFrame

from funcs import clean_currency, create_forecast_recommendations_flagged, create_forecast_recommendations_all, \
    create_time_series, create_pie_chart, create_box_plot, create_geo_location_plot, \
    create_bar_chart_top_rankings, \
    create_bar_chart_bottom_rankings, create_bar_chart_days_analysis, create_line_plot, create_spending_by_location, \
    create_heatmap
import dash
from dash.dependencies import Input, Output, State
from dash import dcc, html, dash_table
import plotly.express as px

import numpy as np
import pandas as pd

import hvplot.pandas

from nbfuncs import nb_classifier_prediction
from csv_3d_test import create_3D_scatter

import matplotlib.pyplot as plt

# (1) Forecasts - Exponential Smoothing and Moving Averages

In [2]:
df = pd.read_excel('data/transactions_2015_2022.xlsx')
df = df.reset_index(drop=True)

In [3]:
avg_df = df.groupby(['Category'])['Amount'].mean().to_frame().reset_index()
avg_df = avg_df.rename(columns={'Amount': 'Average'})

# creating new dataframes for sma and es forecasts
sma_df = df
exp_smooth_df = df

# simple moving average forecast of each category
sma_df = df.groupby(['Category'])['Amount'].rolling(window=4).mean().to_frame().reset_index()
sma_df.columns = ['Category', 'level_1', 'SMA']

most_recent_sma_df = sma_df.groupby('Category').tail(1)

forecasts = pd.merge(most_recent_sma_df, avg_df, on="Category", how="left")
forecasts = forecasts.reindex(columns=['Category', 'level_1', 'Average', 'SMA'])

# exponential smoothing forecast of each category

# define alpha value
alpha = 0.2

# group transactions by category
grouped = exp_smooth_df.groupby('Category')

# create a new empty column for the exponential smoothed amounts

# iterate through each group
for group_name, group_data in grouped:
    # get the indices for the current group
    group_indices = group_data.index

    # perform exponential smoothing on the amount column for the current group
    smoothed = group_data['Amount'].ewm(alpha=alpha).mean()

    # update the smoothed amounts for the current group in the main dataframe
    exp_smooth_df.loc[group_indices, 'ES'] = smoothed

exp_smooth_df = exp_smooth_df.groupby('Category').tail(1)

# merge back with forecast
forecasts = pd.merge(forecasts, exp_smooth_df[['Category', 'ES']], on="Category", how="inner")

forecasts['Flagged_SMA'] = np.where(forecasts['SMA'] > forecasts['Average'], 'Yes', 'No')
forecasts['Flagged_ES'] = np.where(forecasts['ES'] > forecasts['Average'], 'Yes', 'No')

flagged_categories = forecasts[(forecasts['Flagged_SMA'] == 'Yes') & (forecasts['Flagged_ES'] == 'Yes')][
    ['Category', 'Average', 'SMA', 'ES', 'Flagged_SMA', 'Flagged_ES']]

# calculate percentage change from SMA and average to average for every category
forecasts['pct_change_SMA'] = (forecasts['SMA'] - forecasts['Average']) / \
                              forecasts['Average'] * 100
forecasts['pct_change_ES'] = (forecasts['ES'] - forecasts['Average']) / \
                             forecasts['Average'] * 100

# round all int values to 2 decimal places
forecasts[['Average', 'SMA', 'ES', 'pct_change_SMA', 'pct_change_ES']] = forecasts[
    ['Average', 'SMA', 'ES', 'pct_change_SMA', 'pct_change_ES']].round(2)

forecasts = forecasts.drop(columns='level_1')
forecasts = forecasts.reset_index(drop=True)

forecasts

,Category,Average,SMA,ES,Flagged_SMA,Flagged_ES,pct_change_SMA,pct_change_ES
0,Art,80.93,116.02,90.73,Yes,Yes,43.36,12.11
1,Books,39.59,34.43,42.72,No,Yes,-13.04,7.89
2,Car Maintenance,55.38,63.68,57.21,Yes,Yes,14.98,3.30
3,Clothing,80.11,37.15,43.11,No,No,-53.62,-46.19
4,Electronics,76.14,33.33,48.66,No,No,-56.23,-36.10
5,Entertainment,31.41,68.52,64.72,Yes,Yes,118.12,106.02
6,Furniture,99.76,66.60,71.01,No,No,-33.24,-28.81
7,Gas,44.12,57.22,49.64,Yes,Yes,29.70,12.51
8,Groceries,29.88,32.91,30.30,Yes,Yes,10.14,1.41
9,Health Care,21.36,30.07,31.79,Yes,Yes,40.77,48.80


In [4]:
# Create a Panel DataFrame widget
df_widget = pn.widgets.DataFrame(forecasts, name='Forecast Recommendations', autosize_mode='fit_viewport', sortable=True, text_align='center',
                                title='Forecast Recommendations')

df_widget

DataFrame(autosize_mode='fit_viewport', name='Forecast Recommendations'..., text_align='center', value=           Category  ...)

# (2) Naive Bayes Text Classifier (Necessities vs Non-necessities)

In [5]:
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier
from nltk.tokenize import word_tokenize
import plotly.graph_objects as go

import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\achot\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\achot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
df = pd.read_csv('data/transactions.csv', index_col=False)

learn_df = pd.read_csv('data/transactions.csv')
cleaned_df = learn_df.drop(columns=["Address", "City/State", "Zip Code", "Country", "Amount"])  # Remove cols
cleaned_df['Necessity'] = np.where(learn_df['Category'].isin(
    ['Car Insurance', 'Car Loan', 'Car Maintenance', 'Electric Bill', 'Gas', 'Gas Bill', 'Groceries',
     'Health Care', 'Housing', 'Internet Bill']), True, False)

cleaned_df = cleaned_df.loc[:, ~cleaned_df.columns.str.contains('^Unnamed')]  # Removes Unnamed columns

cleaned_df

,Date,Description,Category,Necessity
0,01/04/22,E-Payment,Housing,True
1,01/04/22,UBER EATS SAN FRANCISCO CA,Restaurants,False
2,01/06/22,BT*TUTORS.COM AUSTIN TX,Subscription,False
3,01/09/22,MT FUJI SUSHI JAPANEMAPLEWOOD NJ,Restaurants,False
4,01/10/22,Audible audible.com NJ,Books,False
...,...,...,...,...
329,12/25/22,NEW JERSEY E-Z PASS 888-288-6865 NJ,Parking & Tolls,False
330,12/27/22,STEAMPOWERED.COM,Entertainment,False
331,12/30/22,MACYS SHORT HILLS 0SHORT HILLS NJ,Clothing,False
332,12/30/22,UMI SUSHI 0001 MILLBURN NJ,Restaurants,False


In [7]:
# Define the stop words
stop_words = set(stopwords.words('english'))

# Define the necessary categories
necessary_categories = ['Car Insurance', 'Car Loan', 'Car Maintenance', 'Electric Bill', 'Gas', 'Gas Bill',
                        'Groceries', 'Health Care', 'Housing', 'Internet Bill']

# Clean the text
cleaned_text = []
for text in learn_df['Description']:
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stop words and punctuation
    words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
    cleaned_text.append(words)

# Get the frequency distribution of the words
all_words = []
for words in cleaned_text:
    all_words += words
fd = FreqDist(all_words)

# Define a feature extractor function that returns a dictionary of word frequencies
def document_features(document):
    document_words = set(document)
    features = {}
    for word in fd.keys():
        features['contains({})'.format(word)] = (word in document_words)
    return features

# Create a labeled feature set
featuresets = [(document_features(text), category in necessary_categories) for text, category in
               zip(cleaned_text, learn_df['Category'])]

# Split the data into training and testing sets
train_set, test_set = featuresets[100:], featuresets[:100]

# Train the classifier
classifier = NaiveBayesClassifier.train(train_set)

# Test the classifier
accuracy = nltk.classify.util.accuracy(classifier, test_set)
print('Accuracy:', accuracy)

# Load new data
data_new = df
df_new = pd.DataFrame(data_new)

# Clean the text
cleaned_text_new = []
for text in df_new['Description']:
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stop words and punctuation
    words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
    cleaned_text_new.append(words)

# Get the predictions for the new data
predictions = []
for text in cleaned_text_new:
    predictions.append(classifier.classify(document_features(text)))

# Create a new DataFrame that includes the predicted values
results_df = pd.DataFrame({'Description': df_new['Description'], 'Predicted_Necessity': predictions})
results_df

Accuracy: 0.79


,Description,Predicted_Necessity
0,E-Payment,True
1,UBER EATS SAN FRANCISCO CA,False
2,BT*TUTORS.COM AUSTIN TX,False
3,MT FUJI SUSHI JAPANEMAPLEWOOD NJ,False
4,Audible audible.com NJ,False
...,...,...
329,NEW JERSEY E-Z PASS 888-288-6865 NJ,False
330,STEAMPOWERED.COM,True
331,MACYS SHORT HILLS 0SHORT HILLS NJ,False
332,UMI SUSHI 0001 MILLBURN NJ,False


In [8]:
import panel as pn

# Create a Panel DataFrame widget with pagination
nb_necessities_widget = pn.widgets.DataFrame(
    results_df,
    name='Naive Bayes',
    pagination='remote',
    page_size=10,
    autosize_mode='fit_viewport',
    sortable=True,
    text_align='center'
)

# Display the DataFrame widget with pagination
nb_necessities_widget


DataFrame(autosize_mode='fit_viewport', name='Naive Bayes', text_align='center', value=              ...)

# (3) Monthly Spend by Category

In [15]:
# Read CSV
df = pd.read_excel('data/transactions_2015_2022.xlsx')

# Preprocessing - Removing unnamed columns, currency to float, and converting date column to date type
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Amount'] = df['Amount'].apply(clean_currency).astype('float')
df['Month'] = (df['Date'].dt.year - df['Date'].dt.year.min()) * 12 + df['Date'].dt.month

# Drop original date column
df.drop('Date', axis=1, inplace=True)

# Create dummy variables for each spending category
one_hot_encoded = pd.get_dummies(df['Category'])
new_columns = ['' + column for column in one_hot_encoded.columns]
encoded_df = pd.DataFrame(one_hot_encoded.values, columns=new_columns)

# Drop the 'Category' column from the original DataFrame
df.drop('Category', axis=1, inplace=True)

# Concatenate the original DataFrame with the encoded DataFrame
df = pd.concat([df, encoded_df], axis=1)
df.head(10)


,Description,Amount,Month,Art,Books,Car Maintenance,Clothing,Electronics,Entertainment,Furniture,Gas,Groceries,Health Care,Misc,Parking & Tolls,Restaurants,Subscription,Travel
0,UBER EATS SAN FRANCISCO CA,26.33,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,BT*TUTORS.COM AUSTIN TX,29.99,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,MT FUJI SUSHI JAPANEMAPLEWOOD NJ,17.81,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,AUDIBLE AUDIBLE.COM NJ,15.66,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,UPWORK*-443405899REFSANTA CLARA,92.70,1.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,APPLE STORE R043 R04SHORT HILLS NJ,84.23,1.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6,MCDONALD'S WEST ORANGE NJ,7.46,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7,INDIGO PERSONAL - BOOKS WILMINGTON DE,22.44,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,NORDSTROM SHORT HILLS NJ,61.10,1.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9,THE HOME DEPOT VAUXHALL NJ,36.20,1.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [16]:
# Make Dataframe Pipeline Interactive
idf = pd.DataFrame()
idf['Art'] = np.where(df['Art'] == 1, df['Amount'], 0)
idf['Books'] = np.where(df['Books'] == 1, df['Amount'], 0)
idf['Car Maintenance'] = np.where(df['Car Maintenance'] == 1, df['Amount'], 0)
idf['Clothing'] = np.where(df['Clothing'] == 1, df['Amount'], 0)
idf['Electronics'] = np.where(df['Electronics'] == 1, df['Amount'], 0)
idf['Entertainment'] = np.where(df['Entertainment'] == 1, df['Amount'], 0)
idf['Furniture'] = np.where(df['Furniture'] == 1, df['Amount'], 0)
idf['Gas'] = np.where(df['Gas'] == 1, df['Amount'], 0)
idf['Groceries'] = np.where(df['Groceries'] == 1, df['Amount'], 0)
idf['Health Care'] = np.where(df['Health Care'] == 1, df['Amount'], 0)
idf['Misc'] = np.where(df['Misc'] == 1, df['Amount'], 0)
idf['Parking & Tolls'] = np.where(df['Parking & Tolls'] == 1, df['Amount'], 0)
idf['Restaurants'] = np.where(df['Restaurants'] == 1, df['Amount'], 0)
idf['Subscription'] = np.where(df['Subscription'] == 1, df['Amount'], 0)
idf['Travel'] = np.where(df['Travel'] == 1, df['Amount'], 0)
idf['Amount'] = df['Amount']
idf['Month'] = df['Month'].astype('Int64')
idf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Art              2215 non-null   float64
 1   Books            2215 non-null   float64
 2   Car Maintenance  2215 non-null   float64
 3   Clothing         2215 non-null   float64
 4   Electronics      2215 non-null   float64
 5   Entertainment    2215 non-null   float64
 6   Furniture        2215 non-null   float64
 7   Gas              2215 non-null   float64
 8   Groceries        2215 non-null   float64
 9   Health Care      2215 non-null   float64
 10  Misc             2215 non-null   float64
 11  Parking & Tolls  2215 non-null   float64
 12  Restaurants      2215 non-null   float64
 13  Subscription     2215 non-null   float64
 14  Travel           2215 non-null   float64
 15  Amount           2197 non-null   float64
 16  Month            2197 non-null   Int64  
dtypes: Int64(1), f

In [17]:
#Make month slider
month_slider = pn.widgets.IntSlider(name='Month slider', start=1, end=96, step=1, value=1)
month_slider

IntSlider(end=96, name='Month slider', start=1, value=1)

In [18]:
# Radio Buttons For Spending Categories
spend_categories = pn.widgets.Select(
    name='Spend Categories',
    options=['Art', 'Books', 'Car Maintenance', 'Clothing', 'Electronics',
             'Entertainment', 'Furniture', 'Gas', 'Groceries',
             'Health Care', 'Misc', 'Parking & Tolls', 'Restaurants', 'Subscription', 'Travel'],
    sizing_mode='stretch_width'
)

spend_categories

Select(name='Spend Categories', options=['Art', 'Books', ...], sizing_mode='stretch_width', value='Art')

In [19]:
idf = idf.interactive()
time_series_pipeline = (
    idf[
        (idf['Month'] <= month_slider)
    ]
    .groupby(['Month'])[spend_categories].sum()
    .reset_index()
    .sort_values(by='Month')
    .reset_index(drop=True)
        )

In [20]:
#Check if time series has been created
time_series_pipeline

In [21]:
time_series_plot = time_series_pipeline.hvplot(x = 'Month', y = spend_categories, line_width = 2,
                   
                                               title = 'Time-Series Spending by Category', color='#1B3769')
time_series_plot

In [22]:
df2 = pd.read_excel('data/transactions_2015_2022.xlsx')
df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]
df2['Amount'] = df2['Amount'].apply(clean_currency).astype('float')
df2['Month'] = (df2['Date'].dt.year - df2['Date'].dt.year.min()) * 12 + df2['Date'].dt.month
# Drop original date column
df2.drop('Date', axis=1, inplace=True)
df2.drop('Description', axis=1, inplace=True)
df2.head()

,Amount,Category,Month
0,26.33,Restaurants,1.0
1,29.99,Subscription,1.0
2,17.81,Restaurants,1.0
3,15.66,Books,1.0
4,92.70,Misc,1.0


In [32]:
import pandas as pd
import plotly.graph_objects as go
import panel as pn

# Calculate the average monthly spending per category
df_avg_spending = df2.groupby(['Category', 'Month'])['Amount'].mean().reset_index()

# Pivot the data for plotting
df_pivot = df_avg_spending.pivot(index='Category', columns='Month', values='Amount')
df_pivot = df_pivot.reset_index().fillna(0)

# Set the months as columns and convert to a list
months = df_pivot.columns[1:].astype(int).tolist()

# Reset the index of the pivot table
df_pivot.reset_index(drop=True, inplace=True)

# Create a list to store the radar charts
figs = []

# Iterate over each month
for month in months:
    # Create a radar chart for the current month
    fig = go.Figure()

    # Retrieve the amounts for the current month
    amounts = df_pivot[month].tolist()

    # Add traces for each category
    for category, amount in zip(df_pivot['Category'], amounts):
        fig.add_trace(go.Scatterpolar(
            r=[amount],
            theta=[category],
            fill='toself',
            name=category
        ))

    # Set layout and title for the radar chart
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True
            ),
            angularaxis=dict(
                tickmode='array',
                tickvals=df_pivot.index,
                ticktext=df_pivot['Category']
            )
        ),
        showlegend=True,
        title=f"Radar Chart - Month {month}"
    )

    # Append the radar chart to the list
    figs.append(fig)


# Define the function to update the displayed chart based on the selected month
def update_chart(event):
    month = dropdown.value
    panel_layout[1] = pn.pane.Plotly(figs[month])


# Create the dropdown menu for selecting the month
dropdown = pn.widgets.DiscreteSlider(name='Select Month', options=list(range(len(months))), value=0)

# Connect the update_chart function to the value parameter of the dropdown
dropdown.param.watch(update_chart, 'value')

# Create a panel layout with the dropdown menu and radar chart
panel_layout = pn.Row(dropdown, pn.pane.Plotly(figs[0]))

# Display the panel layout
pn.extension()
panel_layout



Row
    [0] DiscreteSlider(formatter='%d', name='Select Month', options=[0, 1, 2, 3, 4, ...], value=0)
    [1] Plotly(Figure)

# (4) Spending by Day of Week

In [33]:
#Create new df from excel file and combine with original
wdf = pd.read_excel('data/transactions_2015_2022.xlsx')
wdf['Week'] = (wdf['Date'].dt.year - wdf['Date'].dt.year.min()) * 52 + wdf['Date'].dt.week
wdf = wdf.groupby(['Week'])['Amount'].mean()
wdf = wdf.tail(5)
wdf

C:\Users\achot\AppData\Local\Temp\ipykernel_14392\2829411699.py:3: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



Week
412.0     19.078782
413.0    113.139180
414.0     42.573371
415.0     58.869649
416.0     45.088144
Name: Amount, dtype: float64

In [34]:
#Make Week Slider
#week_slider = pn.widgets.IntSlider(name='Week slider', start=1, end=416, step=4, value=1)
#week_slider

In [35]:
#wdf.sort_values(by='Day_of_Week', inplace=True)
wdf = wdf.interactive()
dow_source_bar_pipeline = (
    wdf.groupby(['Week'])['Amount'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Week')  
    .reset_index(drop=True)
)

In [36]:
#Check if data loaded in 
#dow_source_bar_pipeline 

In [37]:
dow_source_bar_plot = dow_source_bar_pipeline.hvplot(kind = 'bar',
                                                    x='Week',
                                                    y = 'Amount',
                                                    title = 'Spending by Last 5 Weeks',color='#1B3769')
dow_source_bar_plot

# (5) Panel Dashboard

In [40]:
# Add plot to dashboard
# Layout using Template
SMA_ES_col = pn.Column('####Spending Forecast Recommendation System', df_widget)
nb_col = pn.Column('####Naive Bayes Text Classifier', nb_necessities_widget, height=50)
template = pn.template.FastListTemplate(
    title='Improving Your Spending Habits with Predictive and Prescriptive Analytics',
    sidebar=[
        pn.pane.Markdown("# Spendalyzer"),
        pn.pane.Markdown("## Bank Statement Analytics"),
        pn.pane.Markdown("###Developed By: Franklin Tan and Ascari Chotoosingh"),
        pn.pane.Markdown("#### A solution that combines budgeting and financial planning, identifying areas of overspending, improving financial literacy, and tracking progress towards financial goals."),
        pn.pane.JPG('https://www.saic.edu/150/sites/default/files/Monopoly.jpg', sizing_mode='scale_both'),
        pn.pane.Markdown("### Settings"),
        month_slider,
        spend_categories
    ],
    main=[
        pn.Column(
            panel_layout,
            pn.Row(spend_categories, width=400),
            pn.Row(time_series_plot.panel(width=700), dow_source_bar_plot.panel(width=700), margin=(0, 25)),
            pn.Row(SMA_ES_col, nb_col, margin=(0, 25))
        )
    ],
    accent_base_color="#1B3769",
    header_background="#1B3769"
)

template.show()



Launching server at http://localhost:61403
